# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [63]:
cities_df = pd.read_csv('WeatherPy_outputs/city_df.csv')
#print(cities_df)
cities_df.columns

Index(['city_name', 'country', 'latitude', 'longitude', 'max_temperature',
       'humidity', 'cloudiness', 'wind speed'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [64]:
gmaps.configure(api_key=gkey)

#add locations and humidity
#converted humidity to a number for heatmap
locations = cities_df[["latitude", "longitude"]]
humidity = cities_df["humidity"].astype(float)
max_humidity = humidity.max()

In [65]:
humidity_heatmap = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=3)
humidity_heatmap.add_layer(heat_layer)
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.
* A max temperature lower than 30 degrees but higher than 20.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.

In [67]:
# #finding perfect weather conditions
#note, temperatures are in degrees Celsius
perfect_weather = cities_df.loc[(cities_df["max_temperature"] > 20) & (cities_df["max_temperature"] < 30)& (cities_df["cloudiness"] == 0), :]
perfect_weather = perfect_weather.dropna(how='any')
perfect_weather.rename(columns = {'city_name':'City', 'country':'Country'}, inplace = True)
perfect_weather.head()

,City,Country,latitude,longitude,max_temperature,humidity,cloudiness,wind speed
29,Jamestown,US,42.0970,-79.2353,22.80,40,0,2.57
30,Nizwá,OM,22.9333,57.5333,24.57,43,0,0.94
33,Cabo San Lucas,MX,22.8909,-109.9124,25.90,63,0,5.14
51,Lompoc,US,34.6391,-120.4579,26.43,76,0,8.05
64,Aripuanã,BR,-9.1667,-60.6333,23.57,93,0,0.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
#store into variable named hotel_df
hotel_df = perfect_weather
hotel_name = []
# # len(hotel_df)
for i,r in hotel_df.iterrows():
    lat = r['latitude']
    lng = r['longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "lodging",
        "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_name.append(response['results'][0]['name'])
    except:
        hotel_name.append("")

hotel_df["Hotel Name"] = hotel_name
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City,Country,latitude,longitude,max_temperature,humidity,cloudiness,wind speed,Hotel Name
29,Jamestown,US,42.0970,-79.2353,22.80,40,0,2.57,Hampton Inn & Suites Jamestown
30,Nizwá,OM,22.9333,57.5333,24.57,43,0,0.94,Orient Apartments
33,Cabo San Lucas,MX,22.8909,-109.9124,25.90,63,0,5.14,Hotel Tesoro Los Cabos
51,Lompoc,US,34.6391,-120.4579,26.43,76,0,8.05,"Holiday Inn Express Lompoc, an IHG Hotel"
64,Aripuanã,BR,-9.1667,-60.6333,23.57,93,0,0.33,


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**r) for i, r in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [75]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
humidity_heatmap.add_layer(markers)
humidity_heatmap.add_layer(hotel_layer)
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))